In [1]:
import geopandas as gpd
import numpy as np

In [2]:
start_time = np.datetime64('2021-06-30T01:00:00.00')
end_time = np.datetime64('2021-07-01T00:00:00.00')

In [3]:
aoi = gpd.read_file('test.json')
aoi

,OBJECTID,Shape_Length,Shape_Area,geometry
0,2,18.534866,21.47133,"POLYGON ((121.25398 48.62437, 121.28882 43.990..."


In [4]:
from wis_stac import wis_stac

In [5]:
catalog = wis_stac(aoi, np.datetime64('2021-06-30T01:00:00.00'),np.datetime64('2021-07-01T00:00:00.00'))

In [6]:
catalog.collections

['era5-land',
 'gpm-imerg-early',
 'gfs_atmos',
 'sentinel-s2-l2a-cogs',
 'landsat-8-l1-c1']

In [7]:
era5_items = catalog.items('era5-land')
era5_items

,id,start_time,end_time,geometry
0,era5-land,2021-06-30 01:00:00,2021-07-01,"POLYGON ((125.88757 48.65921, 125.92241 44.025..."


In [8]:
from leafmap import Map
m=Map()
m.add_gdf(era5_items,layer_name='era5')

In [9]:
m

Map(center=[46.32499302650018, 123.58819695900013], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [3]:
from wis_s3api import era5

In [4]:
ds = era5.from_shp(['2 metre temperature'],start_time=start_time,end_time=end_time,shp='test.json')
ds

<xarray.Dataset>
Dimensions:  (time: 24, lon: 47, lat: 48)
Coordinates:
  * lat      (lat) float32 44.0 44.1 44.2 44.3 44.4 ... 48.3 48.4 48.5 48.6 48.7
  * lon      (lon) float32 121.3 121.4 121.5 121.6 ... 125.6 125.7 125.8 125.9
  * time     (time) datetime64[ns] 2021-06-30T01:00:00 ... 2021-07-01
Data variables:
    t2m      (time, lon, lat) float32 dask.array<chunksize=(23, 47, 48), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [5]:
ds = ds.transpose('time','lat','lon')
ds

<xarray.Dataset>
Dimensions:  (time: 24, lon: 47, lat: 48)
Coordinates:
  * lat      (lat) float32 44.0 44.1 44.2 44.3 44.4 ... 48.3 48.4 48.5 48.6 48.7
  * lon      (lon) float32 121.3 121.4 121.5 121.6 ... 125.6 125.7 125.8 125.9
  * time     (time) datetime64[ns] 2021-06-30T01:00:00 ... 2021-07-01
Data variables:
    t2m      (time, lat, lon) float32 dask.array<chunksize=(23, 48, 47), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [31]:
import rioxarray as rio
xds = rio.open_rasterio('testt2m.nc')
xds

<xarray.DataArray 't2m' (time: 24, y: 48, x: 47)>
array([[[  1939,   1570, ...,   4848,   4434],
        [   448,    780, ...,   5925,   5638],
        ...,
        [  7931,   8093, ...,   7696,   7367],
        [  7955,   8178, ...,   7161,   7076]],

       [[  2328,   1149, ...,   6869,   6456],
        [   914,    492, ...,   7837,   7558],
        ...,
        [  9733,   9685, ...,   9729,   9462],
        [  9912,   9899, ...,   9203,   9211]],

       ...,

       [[-13015, -13505, ...,  -4824,  -5010],
        [-14494, -14287, ...,  -4220,  -4313],
        ...,
        [  1342,   1812, ...,  -2632,  -3021],
        [  1695,   2262, ...,  -2754,  -3025]],

       [[-10434, -10983, ...,    323,    134],
        [-12151, -11852, ...,    929,    876],
        ...,
        [  8116,   8665, ...,    911,    542],
        [  8604,   9188, ...,    652,    450]]], dtype=int16)
Coordinates:
  * time         (time) object 2021-06-30 01:00:00 ... 2021-07-01 00:00:00
  * x            (x) float64 121.3 121.4 121.5 121.6 ... 125.6 125.7 125.8 125.9
  * y            (y) float64 48.7 48.6 48.5 48.4 48.3 ... 44.3 44.2 44.1 44.0
    spatial_ref  int64 0
Attributes:
    lat#long_name:           latitude
    lat#units:               degrees_north
    lat#_FillValue:          nan
    lon#long_name:           longitude
    lon#units:               degrees_east
    lon#_FillValue:          nan
    NETCDF_DIM_EXTRA:        {time}
    NETCDF_DIM_time_DEF:     [24.  4.]
    NETCDF_DIM_time_VALUES:  [1065001. 1065002. 1065003. 1065004. 1065005. 10...
    add_offset:              255.22716820917262
    long_name:               2 metre temperature
    scale_factor:            0.0006959302875823297
    units:                   ('K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K...
    _FillValue:              -32767

In [76]:
rds = xds.isel(time=0)
rds

<xarray.DataArray 't2m' (y: 48, x: 47)>
array([[1939, 1570, 1197, ..., 5382, 4848, 4434],
       [ 448,  780,  756, ..., 6419, 5925, 5638],
       [-415,   75,  201, ..., 6983, 6662, 6367],
       ...,
       [7902, 8133, 8275, ..., 8024, 8105, 7809],
       [7931, 8093, 8376, ..., 7748, 7696, 7367],
       [7955, 8178, 8238, ..., 7201, 7161, 7076]], dtype=int16)
Coordinates:
    time         object 2021-06-30 01:00:00
  * x            (x) float64 121.3 121.4 121.5 121.6 ... 125.6 125.7 125.8 125.9
  * y            (y) float64 48.7 48.6 48.5 48.4 48.3 ... 44.3 44.2 44.1 44.0
    spatial_ref  int64 0
Attributes:
    lat#long_name:           latitude
    lat#units:               degrees_north
    lat#_FillValue:          nan
    lon#long_name:           longitude
    lon#units:               degrees_east
    lon#_FillValue:          nan
    NETCDF_DIM_EXTRA:        {time}
    NETCDF_DIM_time_DEF:     [24.  4.]
    NETCDF_DIM_time_VALUES:  [1065001. 1065002. 1065003. 1065004. 1065005. 10...
    add_offset:              255.22716820917262
    long_name:               2 metre temperature
    scale_factor:            0.0006959302875823297
    units:                   ('K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K...
    _FillValue:              -32767

In [77]:
rds.rio.write_crs(rasterio.crs.CRS.from_epsg(4326),inplace=True)
rds.rio.crs

CRS.from_epsg(4326)

In [87]:
xds3857 = rds.rio.reproject("epsg:3857")
xds3857

<xarray.DataArray 't2m' (y: 56, x: 38)>
array([[1939, 1570, 1254, ..., 5382, 4848, 4434],
       [ 448,  780,  772, ..., 6419, 5925, 5638],
       [-415,   75,   47, ..., 6983, 6662, 6367],
       ...,
       [7931, 8093, 8186, ..., 7748, 7696, 7367],
       [7931, 8093, 8186, ..., 7748, 7696, 7367],
       [7955, 8178, 8117, ..., 7201, 7161, 7076]], dtype=int16)
Coordinates:
  * x            (x) float64 1.35e+07 1.352e+07 ... 1.401e+07 1.402e+07
  * y            (y) float64 6.226e+06 6.212e+06 ... 5.474e+06 5.46e+06
    time         object 2021-06-30 01:00:00
    spatial_ref  int64 0
Attributes:
    lat#long_name:           latitude
    lat#units:               degrees_north
    lat#_FillValue:          nan
    lon#long_name:           longitude
    lon#units:               degrees_east
    lon#_FillValue:          nan
    NETCDF_DIM_EXTRA:        {time}
    NETCDF_DIM_time_DEF:     [24.  4.]
    NETCDF_DIM_time_VALUES:  [1065001. 1065002. 1065003. 1065004. 1065005. 10...
    add_offset:              255.22716820917262
    long_name:               2 metre temperature
    scale_factor:            0.0006959302875823297
    units:                   ('K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K...
    _FillValue:              -32767

In [89]:
xds3857.rio.to_raster("testt2m3857.tif")

In [78]:
rds.rio.to_raster('testt2m.tif')

In [10]:
from leafmap import Map
m=Map()

In [11]:
m.add_raster('testt2m3857.tif',palette='coolwarm',layer_name='t2m3857')
m

ImportError: localtileserver is not installed. Please install it before proceeding. https://github.com/banesullivan/localtileserver

In [118]:
import rasterio

In [111]:
rr = rasterio.open('testt2m3857.tif')

In [112]:
rr.crs

CRS.from_epsg(3857)

In [113]:
rr.shape

(56, 38)